# **Advent of Code 2022**

In [1]:
from __future__ import annotations
import numpy as np
import pandas as pd
from functools import reduce
import copy
from dataclasses import dataclass, field

import warnings
warnings.filterwarnings("ignore")

: 

## **1. Advent**

In [2]:
l = [i.split("\n") for i in open("inputs22/1").read().split("\n\n")]
s = [sum([int(i) for i in sl]) for sl in l]
print(f"Task 1: {max(s)}, Task 2: {sum(sorted(s)[-3:])}")

Task 1: 69795, Task 2: 208437


## **2. Advent**

In [3]:
df = pd.read_csv("inputs22/2", header=None, sep=" ")
t = lambda i: df.apply(lambda row: f(row[0], row[1])[i], axis=1).sum()
def f(opp, you):
    match (opp, you):
        case ("A", "X"): return (3 + 1), (0 + 3)
        case ("B", "X"): return (0 + 1), (0 + 1)
        case ("C", "X"): return (6 + 1), (0 + 2)
        case ("A", "Y"): return (6 + 2), (3 + 1)
        case ("B", "Y"): return (3 + 2), (3 + 2)
        case ("C", "Y"): return (0 + 2), (3 + 3)
        case ("A", "Z"): return (0 + 3), (6 + 2)
        case ("B", "Z"): return (6 + 3), (6 + 3)
        case ("C", "Z"): return (3 + 3), (6 + 1)
print(f"Task 1: {t(0)}, Task 2: {t(1)}")

Task 1: 8392, Task 2: 10116


## **3. Advent**

In [4]:
l = open("inputs22/3").read().split("\n")
t = [(s[:len(s)//2], s[len(s)//2:]) for s in l]
t2 = [l[i:i+3] for i in range(0, len(l), 3)]
scorer = lambda s: [ord(x) - (96 if x.islower() else 38) for x in s]
intersector = lambda t: [set.intersection(*[set(i) for i in p]).pop() for p in t]
f = lambda t: sum(scorer(intersector(t)))
print(f"Task 1: {f(t)}, Task 2: {f(t2)}")

Task 1: 8123, Task 2: 2620


## **4. Advent**

In [5]:
l = [[[int(k) for k in j.split("-")] for j in i.split(",")] for i in open("inputs22/4").read().split("\n")]
splitter = lambda p: (set(range(p[0][0], p[0][1]+1)), set(range(p[1][0], p[1][1]+1)))
f1 = lambda s1, s2: s1.issubset(s2) or s2.issubset(s1)
f2 = lambda s1, s2: (s1 & s2) != set()
r = lambda f: reduce(lambda b, p: f(*splitter(p)) + b, l, 0)
print(f"Task 1: {r(f1)}, Task 2: {r(f2)}")

Task 1: 466, Task 2: 865


## **5. Advent**

In [6]:
crates_s, insts_s = open("inputs22/5").read().split("\n\n")

N = int(crates_s.split(" ")[-2])
C = [[] for _ in range(N)]
for line in crates_s.split("\n")[:-1]:
    for i, ci in enumerate(range(1, 4*N, 4)):
        c = line[ci]
        if c != " ": C[i].insert(0, c)

l = [i.split(" ") for i in insts_s.split("\n")]
I = [[int(sl[i]) for i in [1,3,5]] for sl in l]

split = lambda l, i: (l[:i], l[i:])

def mover(C, inst, mn):
    num_crates, orig, dest = inst[0], inst[1]-1, inst[2]-1
    C[orig], c = split(C[orig], -num_crates)
    C[dest] += reversed(c) if mn == 9000 else c
    return C

mover_reduce = lambda mn: reduce(lambda c, inst: mover(c, inst, mn), I, copy.deepcopy(C))
f = lambda mn: "".join([c[-1] for c in mover_reduce(mn)])
print(f"Task 1: {f(9000)}, Task 2: {f(9001)}")

Task 1: LBLVVTVLP, Task 2: TPFFBDRJD


## **6. Advent**

In [7]:
s = open("inputs22/6").read()
f = lambda n: [i+n for i in range(len(s)-n) if len(set(s[i:i+n])) == n][0]
print(f"Task 1: {f(4)}, Task 2: {f(14)}")

Task 1: 1198, Task 2: 3120


In [8]:
s = open("inputs22/7t").read().split("$ ")
ss = [i.split("\n") for i in s]
ss[-1].append("hack")
sss = [j[:-1] for j in ss[1:]]

class Node:
    parent: 'Dir' | None
    name: str
    level: int

    def __init__(self, parent, name, level):
        self.parent = parent
        self.name = name
        self.level = level

class File(Node):
    size: int

    def __init__(self, parent, name, size, level):
        self.size = size
        super().__init__(parent, name, level)

    def __repr__(self):
        return f"{self.name} (file, size={self.size})"

class Dir(Node):
    children: dict[str, Node]
    def __init__(self, parent, name, children, level):
        self.children = children
        super().__init__(parent, name, level)

    def add_dir(self, Dir):
        if Dir.name not in self.children:
            self.children[Dir.name] = Dir

    def add_file(self, File):
        if File.name not in self.children:
            self.children[File.name] = File

    def get_subdir(self, name):
        if name in self.children:
            return self.children[name]
        else:
            print("does this happen?")

    def get_size(self):
        total = 0
        for c in self.children:
            if isinstance(c, File): total += File.size
            if isinstance(c, Dir): total += Dir.get_size()
        return total

    def __repr__(self):
        string = f"{self.name} (dir)"
        for c in self.children:
            string += "\n- " + " " * (self.level)
            string += c.__repr__()
        return string
        

ROOT = Dir(name='', parent=None, level=0, children={})
FS = ROOT

for i, line in enumerate(sss):
    comargs, *outs = line
    com, *args = comargs.split(" ")
    match com:
        case "cd":
            match args[0]:
                case "..": FS = FS.parent
                case "/": FS = ROOT
                case x: FS = FS.get_subdir(x)

        case "ls":
            for out in outs:
                match out.split(" "):
                    case ("dir", name):
                        nd = Dir(name=name, parent=FS, level=FS.level+1, children={})
                        # print("123", nd.children)
                        FS.add_dir(nd)
                        # print("456", FS.children[nd.name].children)

                    case (size, name):
                        FS.add_dir(File(name=name, size=size, parent=FS, level=FS.level+1))


ROOT.get_size()

0

In [9]:
"123/".split("/")

['123', '']

In [10]:
from enum import Enum


class l(Enum):
    H = 1
    T = 2
    HT = 3

l.H
l.T

import numpy as np

np.random.choice([l.H, l.T], size=10)

array([<l.H: 1>, <l.H: 1>, <l.H: 1>, <l.H: 1>, <l.H: 1>, <l.T: 2>,
       <l.H: 1>, <l.H: 1>, <l.T: 2>, <l.H: 1>], dtype=object)

In [11]:
# s = [i.split(" ") for i in open("inputs22/8t").read().split("\n")]
# ss = [[x, int(y)] for x, y in s]

# G = np.zeros((1000,1000,3), dtype=bool)

# # G[500,500] = 1
# # i, j = 
# Hi, Hj = 0,0
# Ti, Tj = 0,0

# def compute_consequence(H, T):
    


# for step in ss:
#     match step:
#         case "R", i:
#             Hj += 1



In [12]:
I = [i.split(" ") for i in open("inputs22/9").read().split("\n")]

X = 1
c = 0
C = [X]

print(len(I))
print(len(I) )


for inst in I:
    match inst:
        case ["noop"]:
            C += [X]
        case "addx", i:
            C += [X]
            C += [X]
            X += int(i)
        case _: print("problem")
C += [X]

# print(len(C))
# print(C[20], C[:20])
l = [C[20], C[60], C[100], C[140], C[180], C[220]]
print(l)
l = [C[20]*20, C[60]*60, C[100]*100, C[140]*140, C[180]*180, C[220]*220]

# C[219:]
print(sum(l))

138
138
[17, 17, 7, 21, 21, 17]
12520


In [26]:
A = np.array(C[1:-1]).reshape(-1, 40)
IDX = np.vstack(list(np.arange(40))*6).reshape(-1,40)

R = np.abs(A - IDX) <= 1

for row in R:
    for el in row:
        if el: 
            print("█", end="")
        else:
            print(" ", end="")
    print()




████ █  █ ███  ████ ███    ██  ██  █    
█    █  █ █  █    █ █  █    █ █  █ █    
███  ████ █  █   █  █  █    █ █    █    
█    █  █ ███   █   ███     █ █ ██ █    
█    █  █ █    █    █    █  █ █  █ █    
████ █  █ █    ████ █     ██   ███ ████ 
